In [1]:
from utils.init import reproducibility, clean_memory_get_device

device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [2]:
from data.transforms import get_tensorise_h_flip_transform
from data.constants import DEFAULT_REFLACX_LABEL_COLS
from data.paths import MIMIC_EYE_PATH
from data.datasets import ReflacxObjectDetectionWithFixations
from torch.utils.data import DataLoader
from data.datasets import collate_fn
from data.load import seed_worker, get_dataloader_g
import torch

print(torch.cuda.memory_summary())

dataset_params_dict = {
    "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
    # "with_clinical": model_setup.use_clinical,
    "bbox_to_mask": False,
    "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
}

train_dataset = ReflacxObjectDetectionWithFixations(
        **dataset_params_dict, split_str="train", transforms=get_tensorise_h_flip_transform(train=False), with_fixations=True
)

train_dataloader =  DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=collate_fn,
    worker_init_fn=seed_worker,
    generator=get_dataloader_g(0),
)
print(torch.cuda.memory_summary())



|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [3]:
from models.components.feature_extractors import ImageFeatureExtractor
from models.components.fusors import NoActionFusor
from models.components.task_performers import ObjectDetectionParameters, ObjectDetectionPerformer, HeatmapGeneratorParameters, HeatmapGenerator
from models.frameworks import ExtractFusePerform
from models.backbones import get_normal_backbone
from models.setup import ModelSetup
from data.constants import DEFAULT_REFLACX_LABEL_COLS
import torch.nn as nn

In [4]:
setup = ModelSetup(image_size=256)
backbone = get_normal_backbone(setup)
image_extractor = ImageFeatureExtractor(backbone)
fusor = NoActionFusor()


obj_params = ObjectDetectionParameters(image_size=setup.image_size)
obj_performer = ObjectDetectionPerformer(
    obj_params,
    image_extractor.backbone.out_channels,
    len(DEFAULT_REFLACX_LABEL_COLS) + 1
)

fix_params = HeatmapGeneratorParameters(input_channel=backbone.out_channels, decoder_channels=[64, 32, 16, 8, 1 ]) # the output should be just one channel.
fix_performer = HeatmapGenerator(
    params= fix_params,
)

model = ExtractFusePerform(
    feature_extractors=nn.ModuleDict({"image": image_extractor}),
    fusor=fusor,
    task_performers=nn.ModuleDict({"object-detection": obj_performer, "fixation-generation": fix_performer }),
)
model.to(device)

print(torch.cuda.memory_summary())


Using pretrained backbone. mobilenet_v3
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   22419 KB |   22419 KB |   22419 KB |       0 B  |
|       from large pool |   17936 KB |   17936 KB |   17936 KB |       0 B  |
|       from small pool |    4483 KB |    4483 KB |    4483 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |   22419 KB |   22419 KB |   22419 KB |       0 B  |
|       from large pool 

In [5]:
def map_dict_elements_to_device(x, device):
    for k in x.keys():
        try:
            x[k] = x[k].to(device) 
        except:
            pass
    
    return x

In [6]:
data = next(iter(train_dataloader))
inputs, targets = train_dataset.prepare_input_from_data(data)
inputs, targets = model.prepare(inputs, targets)
inputs = map_dict_elements_to_device(inputs, device)
targets = [ map_dict_elements_to_device(t, device) for t in targets]

print(torch.cuda.memory_summary())

c:\Users\mike8\Document\GitHub\MIMIC-Eye-applications\data\datasets.py:437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes_df[k] = ellipse_df[


after preparation, the size is torch.Size([1, 256, 256])
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   25753 KB |   25753 KB |   25753 KB |       0 B  |
|       from large pool |   21008 KB |   21008 KB |   21008 KB |       0 B  |
|       from small pool |    4745 KB |    4745 KB |    4745 KB |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |   25753 KB |   25753 KB |   25753 KB |       0 B  |
|      

In [7]:
model.train()
outputs = model(inputs, targets=targets)

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:1320: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
